In [1]:
import os, time, torch

In [2]:
%run ../utilities/curl_utilities.ipynb
%run ../utilities/postgres_utilities.ipynb

In [3]:
model_name = "GritLM/GritLM-7B"
number_data_rows = 100

In [7]:
connect_db()

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

#Note that model will not load without reducing precision to float16
pipe = pipeline("text-generation", model=model_name, trust_remote_code=False, torch_dtype=torch.float16)

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [01:07<00:00, 22.59s/it]
Device set to use cuda:0


In [8]:
sql = f"select description from film limit {number_data_rows}"
film_descriptions = execute_sql_results(sql)

In [10]:
messages = [
    {"role": "user", "content": "Expand this movie description: A fateful story of a dog and a wombat who must wrestle a lion."},
]

results = pipe(messages, max_length=100)
results

[{'generated_text': [{'role': 'user',
    'content': 'Expand this movie description: A fateful story of a dog and a wombat who must wrestle a lion.'},
   {'role': 'assistant',
    'content': '"A Fateful Story of a Dog and a Wombat Who Must Wrestle a Lion" is a heartwarming and action-packed adventure film that tells the tale of two unlikely friends who must come together to save their home and their lives.\n\nThe story begins with a young dog'}]}]

In [11]:
test_description = "Starting online generative test using local model"
print(test_description)
print(len(test_description) * "-") 
question_response_pairs = list()
tic = time.perf_counter()
print("Texts generated ...", end=" ")

#Setup template for prompt
messages = [
    {"role": "user", "content": ""},
]

generated_tokens_running_total = 0
for index, film in enumerate(film_descriptions):
    description = film[0]

    #messages[0]['content'] = f"Who are the main characters in this sentence: {description}"
    messages[0]['content'] = f"Expand this movie description: {description}"

    response = pipe(messages, max_new_tokens=50)
    generated_text = response[0]['generated_text'][1]['content']
    assert len(generated_text) > 0

    question_response_pairs.append(f"{messages[0]['content']}:{generated_text}")

    if index % 25 == 0:
        print(f"{index}", end=" ")

assert len(question_response_pairs) == number_data_rows
print(f"{index+1}", end=" ")
toc = time.perf_counter()
print("")
print(f"Time to generate text for {number_data_rows} rows: {toc - tic:0.4f} seconds")
print(len(test_description) * "-") 

Starting online generative test using local model
-------------------------------------------------
Texts generated ... 0 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


25 50 75 100 
Time to generate text for 100 rows: 182.3374 seconds
-------------------------------------------------


In [12]:
close_db()